In [1]:
from coursea import *
await init()

2024-05-18 09:33:48,317 I 2042631 cfg4py.core:update_config:280 | configuration is
alpha: {data_home: ~/zillionare/alpha/data, tts_server: 'http://127.0.0.1:5002/api/tts?'}
backtest: {url: 'http://192.168.100.114:7080/backtest/api/trade/v0.5/'}
influxdb: {bucket_name: zillionare, enable_compress: true, max_query_size: 5000, org: zillionare,
  token: hwxHycJfp_t6bCOYe2MhEDW4QBOO4FDtgeBWnPR6bGZJGEZ_41m_OHtTJFZKyD2HsbVqkZM8rJNkMvjyoXCG6Q==,
  url: 'http://192.168.100.101:58086'}
notify: {dingtalk_access_token: 58df072143b52368086736cb38236753073ccde6537650cad1d5567747803563,
  keyword: trader}
pluto: {store: ~/zillionare/pluto/store}
redis: {dsn: 'redis://192.168.100.101:56379'}
tasks: {pooling: false, wr: false}

2024-05-18 09:33:48,322 I 2042631 /home/aaron/miniconda3/envs/coursea/lib/python3.8/site-packages/omicron/dal/cache.py:init:94 | init redis cache...
2024-05-18 09:33:48,340 I 2042631 /home/aaron/miniconda3/envs/coursea/lib/python3.8/site-packages/omicron/dal/cache.py:init:124 | 

init securities done


In [41]:
code = "300721.XSHE"
end = datetime.date(2024, 5, 8)
bars = await Stock.get_bars(code, 40, FrameType.DAY, end=end)
cs = Candlestick(bars, ma_groups=[])
cs.plot()

[0.7, 0.15, 0.15]


In [42]:
# 示例2
def feature_washout(bars, threshold=0.05):
    """返回在bars中最后一次洗盘结束的位置，-1表示最后一个bar,
        0表示不存在洗盘模式
    """
    close = bars["close"]
    opn = bars["open"]
    truerange = np.maximum(np.abs(close[1:] - close[:-1]), 
                           np.abs(opn-close)[1:]) 
    # 百分比化
    tr = truerange / close[:-1]
    sign = (opn < close)[1:] * 2 - 1
    signed_tr = tr * sign
    
    print(np.round(signed_tr, 2))
    
feature_washout(bars)

[ 0.01  0.03  0.02  0.1  -0.02  0.03 -0.02 -0.    0.02 -0.01  0.01 -0.04
 -0.02 -0.04  0.02  0.03  0.04  0.04 -0.02 -0.06  0.04 -0.04 -0.    0.2
 -0.14 -0.19  0.09 -0.03  0.01 -0.02  0.02  0.05  0.05  0.01  0.09  0.02
  0.08  0.2   0.06]


In [10]:
# 高开洗盘

code = "605303.XSHG"
end = datetime.date(2024, 1, 17)
bars = await Stock.get_bars(code, 40, FrameType.DAY, end=end)
cs = Candlestick(bars, ma_groups=[], width=600)
cs.plot()

[0.7, 0.15, 0.15]


除了使用$np.maximimum$这种 ufunc 之外，实际上$np.max$也可以用来完成这项任务，只是我们需要先将数组$A$和$B$堆叠成一个矩阵：

In [37]:
# 示例2
A = np.arange(4)
B = np.arange(3, 7)
C = np.arange(8, 4, -1)

Z = np.vstack((A,B,C))

# 通过np.max求每列最大值
r1 = np.max(Z, axis=0)

# 通过np.maximum求最大值
r2 = np.maximum.reduce([A, B, C])

np.array_equal(r1, r2)

True

In [ ]:
    # xpflag = [0] * 2
    # for item in sliding_window_view(binned, window_shape = 3):
    #     if np.array_equal([-1, -1, 1], item):
    #         xpflag.append(1)
    #     else:
    #         xpflag.append(0)

In [44]:
# 示例3
from numpy.lib.stride_tricks import sliding_window_view
def feature_washout(bars, threshold=0.05):
    """返回在bars中最后一次洗盘结束的位置，-1表示最后一个bar,
        0表示不存在洗盘模式
    """
    close = bars["close"]
    opn = bars["open"]
    truerange = np.maximum(np.abs(close[1:] - close[:-1]), 
                           np.abs(opn-close)[1:]) 
    # 百分比化
    tr = truerange / close[:-1]
    sign = (opn < close)[1:] * 2 - 1
    signed_tr = tr * sign
    
    encoded = np.select([signed_tr > threshold, 
                    signed_tr < -threshold], 
                    [1, -1], 0)

    washouts = []
    for i, patten in enumerate(sliding_window_view(encoded, window_shape = 4)):
        if np.array_equal(patten, [1, -1, -1, 1]):
            washouts.append(i)

    return washouts
feature_washout(bars)

[23]